In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
import os
from keras import backend as K
from keras.layers import Conv2D, Dropout, LSTM, BatchNormalization, Input,Activation, MaxPool2D, Flatten, Dense,TimeDistributed
from keras.models import Model, load_model
from keras import metrics 
import random

/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VIDEOS_DIR = './Videos/'
IMAGES_DIR = './Images/'

In [3]:
classes = ['Kicking', 'Riding-Horse', 'Running', 'SkateBoarding', 'Swing-Bench', 'Lifting', 'Swing-Side', 'Walking', 'Golf-Swing']
classes

['Kicking',
 'Riding-Horse',
 'Running',
 'SkateBoarding',
 'Swing-Bench',
 'Lifting',
 'Swing-Side',
 'Walking',
 'Golf-Swing']

In [4]:
class_to_index = {}
for i in range(len(classes)):
    class_to_index[classes[i]] = i
class_to_index

{'Golf-Swing': 8,
 'Kicking': 0,
 'Lifting': 5,
 'Riding-Horse': 1,
 'Running': 2,
 'SkateBoarding': 3,
 'Swing-Bench': 4,
 'Swing-Side': 6,
 'Walking': 7}

In [5]:
videos = []
for x in classes:
    videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
videos

[['014',
  '009',
  '005',
  '011',
  '010',
  '003',
  '012',
  '006',
  '013',
  '004',
  '016',
  '001',
  '007',
  '002',
  '017',
  '015'],
 ['009', '005', '010', '003', '006', '004', '001', '007', '008', '002'],
 ['009', '005', '010', '006', '004', '001', '007', '008', '002'],
 ['009', '005', '010', '003', '006', '004', '001', '007', '008', '002'],
 ['014',
  '009',
  '005',
  '011',
  '010',
  '003',
  '012',
  '006',
  '013',
  '004',
  '016',
  '001',
  '007',
  '008',
  '002',
  '017',
  '015'],
 ['005', '003', '004', '001', '002'],
 ['009', '005', '011', '010', '003', '006', '004', '001', '007', '008', '002'],
 ['014',
  '009',
  '005',
  '011',
  '010',
  '018',
  '003',
  '012',
  '006',
  '013',
  '004',
  '016',
  '001',
  '019',
  '007',
  '008',
  '002',
  '017',
  '015'],
 ['014',
  '009',
  '005',
  '011',
  '010',
  '003',
  '012',
  '006',
  '013',
  '004',
  '001',
  '007',
  '008',
  '002']]

### Function Block

In [54]:
def permute(X,Y):
    train_size = X.shape[0]
    permutation_train = np.random.permutation(train_size)
    X = X[permutation_train]
    Y = Y[permutation_train]
    return X,Y

def load_image(path,image_size):
    image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    return image

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def model_predict(model,images):
    output = K.function([model.layers[0].input,K.learning_phase()],
                        [model.layers[13].output])
    return output([images,0])[0]

def pad(X_cnn,max_len):
    features_len = X_cnn.shape[1]
    length = X_cnn.shape[0]
    X_cnn = list(X_cnn)
    pad_arr = [0 for i in range(features_len)]
    for i in range(max_len-length):
        X_cnn.append(pad_arr)
    return np.array(X_cnn)

def evaluate(X_test,Y_test,model):
    count = 0
    for i in range(len(X_test)):
        pred = model.predict(X_test[i])
        max_pred = [np.argmax(i) for i in pred]
        counts = np.bincount(max_pred)
        class_pred = np.argmax(counts)
        #class_pred = max_pred
        actual = np.argmax(Y_test[i])
        #print("Max Preds time", max_pred)
        #print("Pred",classes[class_pred],"Actual",classes[actual])
        #print()
        if class_pred == actual:
            count += 1
    return float(count)/float(len(Y_test)) * 100.0 

### Build Dataset

In [53]:
def build_dataset_for_lstm_strided(image_size, stride = 10, max_len = 40):
    
    model = load_model('models/Conv/17epochs_valacc_94.h5')
    
    X_train_images = []
    Y_train_images = []
    X_test_images = []
    Y_test_images = []
    
    test_videos = [['002', '009'], ['005', '010'], ['007'], \
                   ['003'], ['006', '012'], ['004'], ['008'], ['004', '012'], ['001', '013']]
    
    for i in range(len(classes)):
        cls = classes[i]
        #test_video = random.randint(0,len(videos[i])-1)
        test = test_videos[i] 
        #print("Selected Video for test is",[videos[i][test_video] for test_video in test])

        for j in range(len(videos[i])):
            vid = videos[i][j]
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            
            filelist = sorted(list(os.listdir(image_r)))
            X_train_images_class = []
            
            for file in filelist:
                if file.endswith(".png"):
                    image = load_image(image_r+file,image_size)
                    X_train_images_class.append(image)
            X_cnn = model_predict(model,np.array(X_train_images_class))
            #print(X_cnn.shape)
            
            del X_train_images_class
            X_test_frames = []                                
            for k in range(0,X_cnn.shape[0],stride):
                lower = k
                upper = min(X_cnn.shape[0],k+max_len)
                if upper == X_cnn.shape[0]:
                    if vid not in test:                
                        X_train_images.append(pad(X_cnn[lower:upper],max_len))
                        Y_train_images.append(i)
                    else:
                        X_test_frames.append(pad(X_cnn[lower:upper],max_len))
                        X_test_images.append(np.array(X_test_frames))        
                        Y_test_images.append(i)
                    #print("Padded frames" , lower , "to" , upper)
                    break
                else:
                    if vid not in test:                
                        X_train_images.append(X_cnn[lower:upper])
                        Y_train_images.append(i)
                    else:
                        X_test_frames.append(X_cnn[lower:upper])
                    #print("Added frames" , lower , "to" , upper)
                    
            print("Processed",videos[i][j],"of","class",classes[i])

        #X_test_images.append(np.array(X_test_frames))        
        #Y_test_images.append(i)
    return X_train_images,Y_train_images,X_test_images,Y_test_images

### Conv Model

In [8]:
def build_model(input_shape):
    X_input = Input(input_shape, name = "Input")
    
    X = BatchNormalization(name = 'BatchNorm_1')(X_input)
    X = Conv2D(32, (7, 7), strides = (5, 5), name="Conv_1a", padding="same")(X)
    X = Activation('relu')(X)
    
    X = Conv2D(32, (3, 3), name = "Conv_1b", padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPool2D((2, 2), name = "Pool_1")(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(32, (3, 3), name ="Conv_2", padding = "same")(X)
    X = Activation('relu')(X)
    X = MaxPool2D((4, 4), name = "Pool_2")(X)
    X = Dropout(0.2)(X)
    
    X = Conv2D(8,(1,1), name='Conv_1x1')(X)
    
    X = Flatten()(X)
    X = Dropout(0.2)(X)
    
    return Model(X_input, X)

In [9]:
def CNN_model(prev_model):
    X = Dense(9,activation='softmax',name='final')(prev_model.output)
    return Model(prev_model.input, X)

In [10]:
model = build_model((172, 172, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 172, 172, 3)       0         
_________________________________________________________________
BatchNorm_1 (BatchNormalizat (None, 172, 172, 3)       12        
_________________________________________________________________
Conv_1a (Conv2D)             (None, 35, 35, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Conv_1b (Conv2D)             (None, 35, 35, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Pool_1 (MaxPooling2D)        (None, 17, 17, 32)        0         
__________

In [11]:
cnn = CNN_model(model)
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 172, 172, 3)       0         
_________________________________________________________________
BatchNorm_1 (BatchNormalizat (None, 172, 172, 3)       12        
_________________________________________________________________
Conv_1a (Conv2D)             (None, 35, 35, 32)        4736      
_________________________________________________________________
activation_1 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Conv_1b (Conv2D)             (None, 35, 35, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 35, 35, 32)        0         
_________________________________________________________________
Pool_1 (MaxPooling2D)        (None, 17, 17, 32)        0         
__________

In [12]:
cnn.compile(loss='categorical_crossentropy', 
            metrics=['accuracy'], 
            optimizer='adam')

### RNN Model

In [21]:
def rnn_model(input_shape):
    X_input = Input(input_shape)
    X = LSTM(32, return_sequences=True)(X_input)
    X = Dropout(0.3)(X)
    X = LSTM(32, return_sequences=False)(X)
    X = Dropout(0.3)(X)
    X = Dense(9,activation='softmax')(X)
    return Model(X_input, X)

In [26]:
rnn = rnn_model((30,128))
rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 30, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 32)            20608     
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 32)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 297       
Total params: 29,225
Trainable params: 29,225
Non-trainable params: 0
_________________________________________________________________


In [27]:
rnn.compile(loss='categorical_crossentropy', 
            metrics=['accuracy'], 
            optimizer='adam')

### Data Loader

In [47]:
def data_loader(stride=10, max_len=40):
    X_train = None
    Y_train = None
    X_test = None
    Y_test = None
    try:
        os.path.exists('Numpy/LSTM_Strided/train_X_'+str(stride)+'_'+str(max_len)+'.npy')
        X_train = np.load('Numpy/LSTM_Strided/train_X_'+str(stride)+'_'+str(max_len)+'.npy')
        Y_train = np.load('Numpy/LSTM_Strided/train_Y_'+str(stride)+'_'+str(max_len)+'.npy')
        X_test = np.load('Numpy/LSTM_Strided/test_X_'+str(stride)+'_'+str(max_len)+'.npy')
        Y_test = np.load('Numpy/LSTM_Strided/test_Y_'+str(stride)+'_'+str(max_len)+'.npy')
    except FileNotFoundError:
        X_train,Y_train,X_test,Y_test = build_dataset_for_lstm_strided((172,172), stride, max_len)
        X_train = np.array(X_train)
        X_test = np.array(X_test)
        Y_train = convert_to_one_hot(np.array(Y_train),9)
        Y_test = convert_to_one_hot(np.array(Y_test),9)

        np.save('Numpy/LSTM_Strided/train_X_'+str(stride)+'_'+str(max_len)+'.npy',X_train)
        np.save('Numpy/LSTM_Strided/train_Y_'+str(stride)+'_'+str(max_len)+'.npy',Y_train)
        np.save('Numpy/LSTM_Strided/test_X_'+str(stride)+'_'+str(max_len)+'.npy',X_test)
        np.save('Numpy/LSTM_Strided/test_Y_'+str(stride)+'_'+str(max_len)+'.npy',Y_test)
    print("Training")    
    print("Shape X",X_train.shape)
    print("Shape Y",Y_train.shape)
    print()
    print("Test")
    print("Shape X",X_test.shape)
    print("Shape Y",Y_test.shape)
    return X_train, Y_train, X_test, Y_test

In [18]:
X_train, Y_train, X_test, Y_test = data_loader(stride=7, max_len=40)

Padded frames 0 to 23
Processed 014 of class Kicking
Padded frames 0 to 22
Processed 009 of class Kicking
Padded frames 0 to 23
Processed 005 of class Kicking
Padded frames 0 to 23
Processed 011 of class Kicking
Padded frames 0 to 23
Processed 010 of class Kicking
Padded frames 0 to 23
Processed 003 of class Kicking
Padded frames 0 to 23
Processed 012 of class Kicking
Padded frames 0 to 23
Processed 006 of class Kicking
Padded frames 0 to 23
Processed 013 of class Kicking
Padded frames 0 to 23
Processed 004 of class Kicking
Padded frames 0 to 23
Processed 016 of class Kicking
Padded frames 0 to 23
Processed 001 of class Kicking
Padded frames 0 to 23
Processed 007 of class Kicking
Padded frames 0 to 23
Processed 002 of class Kicking
Padded frames 0 to 23
Processed 017 of class Kicking
Padded frames 0 to 23
Processed 015 of class Kicking
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Padded frames 30 to 58
Processed 009 of class Riding-Horse
Added frames 0 to 30
Added f

Padded frames 0 to 3
Processed 003 of class Swing-Side
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Added frames 30 to 60
Added frames 40 to 70
Padded frames 50 to 75
Processed 006 of class Swing-Side
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Added frames 30 to 60
Added frames 40 to 70
Padded frames 50 to 75
Processed 004 of class Swing-Side
Added frames 0 to 30
Padded frames 10 to 35
Processed 001 of class Swing-Side
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Added frames 30 to 60
Added frames 40 to 70
Padded frames 50 to 75
Processed 007 of class Swing-Side
Padded frames 0 to 14
Processed 008 of class Swing-Side
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Added frames 30 to 60
Added frames 40 to 70
Padded frames 50 to 75
Processed 002 of class Swing-Side
Added frames 0 to 30
Added frames 10 to 40
Added frames 20 to 50
Added frames 30 to 60
Added frames 40 to 70
Added frames 50 to 80
Added frames 60 

In [19]:
X_train_rnn,Y_train_rnn = permute(X_train,Y_train)

Training
Shape X (456, 30, 128)
Shape Y (456, 9)

Test
Shape X (14,)
Shape Y (14, 9)


In [20]:
[i.shape for i in X_test]

[(1, 30, 128),
 (1, 30, 128),
 (4, 30, 128),
 (2, 30, 128),
 (5, 30, 128),
 (5, 30, 128),
 (3, 30, 128),
 (3, 30, 128),
 (11, 30, 128),
 (1, 30, 128),
 (4, 30, 128),
 (9, 30, 128),
 (1, 30, 128),
 (4, 30, 128)]

## Training

In [55]:
def search():
    results = {}
    histories = {}
    strides = [5, 7, 11, 13]
    lengths = [20, 30, 40, 50]
    for stride in strides:
        for length in lengths:
            rnn = rnn_model((length,128))
            X_train, Y_train, X_test, Y_test = data_loader(stride, length)
            X_train_rnn,Y_train_rnn = permute(X_train,Y_train)
            rnn.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
            histories[str(stride) + '_' + str(length)] = rnn.fit(X_train_rnn, Y_train_rnn, epochs=100, batch_size = X_train_rnn.shape[0], validation_split=0.2)
            rnn.save('models/LSTM_Strided/100ep_double_LSTM_dropout_'+ str(stride) + '_' + str(length)+'.h5')
            results[str(stride) + '_' + str(length)] = evaluate(X_test, Y_test, rnn)
    return results, histories

In [56]:
import time
start = time.time()
results, histories = search()
print(time.time() - start)

Training
Shape X (957, 20, 128)
Shape Y (957, 9)

Test
Shape X (14,)
Shape Y (14, 9)
Train on 765 samples, validate on 192 samples
Epoch 1/100
765/765 [==============================] - 2s 2ms/step - loss: 2.1601 - acc: 0.1869 - val_loss: 2.0984 - val_acc: 0.2812
Epoch 2/100
765/765 [==============================] - 0s 425us/step - loss: 2.1045 - acc: 0.2654 - val_loss: 2.0434 - val_acc: 0.3750
Epoch 3/100
765/765 [==============================] - 0s 424us/step - loss: 2.0562 - acc: 0.3150 - val_loss: 1.9897 - val_acc: 0.4479
Epoch 4/100
765/765 [==============================] - 0s 392us/step - loss: 1.9905 - acc: 0.3987 - val_loss: 1.9378 - val_acc: 0.5000
Epoch 5/100
765/765 [==============================] - 0s 380us/step - loss: 1.9369 - acc: 0.4641 - val_loss: 1.8877 - val_acc: 0.5312
Epoch 6/100
765/765 [==============================] - 0s 394us/step - loss: 1.8947 - acc: 0.5085 - val_loss: 1.8385 - val_acc: 0.5833
Epoch 7/100
765/765 [==============================] - 0s 377

Epoch 61/100
765/765 [==============================] - 0s 394us/step - loss: 0.3497 - acc: 0.9608 - val_loss: 0.2837 - val_acc: 0.9740
Epoch 62/100
765/765 [==============================] - 0s 366us/step - loss: 0.3321 - acc: 0.9582 - val_loss: 0.2725 - val_acc: 0.9740
Epoch 63/100
765/765 [==============================] - 0s 353us/step - loss: 0.3170 - acc: 0.9647 - val_loss: 0.2618 - val_acc: 0.9740
Epoch 64/100
765/765 [==============================] - 0s 348us/step - loss: 0.3171 - acc: 0.9621 - val_loss: 0.2513 - val_acc: 0.9740
Epoch 65/100
765/765 [==============================] - 0s 349us/step - loss: 0.2935 - acc: 0.9634 - val_loss: 0.2420 - val_acc: 0.9740
Epoch 66/100
765/765 [==============================] - 0s 349us/step - loss: 0.2851 - acc: 0.9673 - val_loss: 0.2334 - val_acc: 0.9740
Epoch 67/100
765/765 [==============================] - 0s 355us/step - loss: 0.2811 - acc: 0.9660 - val_loss: 0.2251 - val_acc: 0.9740
Epoch 68/100
765/765 [==========================

Processed 010 of class Walking
Processed 018 of class Walking
Processed 003 of class Walking
Processed 012 of class Walking
Processed 006 of class Walking
Processed 013 of class Walking
Processed 004 of class Walking
Processed 016 of class Walking
Processed 001 of class Walking
Processed 019 of class Walking
Processed 007 of class Walking
Processed 008 of class Walking
Processed 002 of class Walking
Processed 017 of class Walking
Processed 015 of class Walking
Processed 014 of class Golf-Swing
Processed 009 of class Golf-Swing
Processed 005 of class Golf-Swing
Processed 011 of class Golf-Swing
Processed 010 of class Golf-Swing
Processed 003 of class Golf-Swing
Processed 012 of class Golf-Swing
Processed 006 of class Golf-Swing
Processed 013 of class Golf-Swing
Processed 004 of class Golf-Swing
Processed 001 of class Golf-Swing
Processed 007 of class Golf-Swing
Processed 008 of class Golf-Swing
Processed 002 of class Golf-Swing
Training
Shape X (781, 30, 128)
Shape Y (781, 9)

Test
Shap

Epoch 54/100
624/624 [==============================] - 0s 524us/step - loss: 0.3345 - acc: 0.9712 - val_loss: 0.2161 - val_acc: 0.9873
Epoch 55/100
624/624 [==============================] - 0s 535us/step - loss: 0.3066 - acc: 0.9744 - val_loss: 0.2053 - val_acc: 0.9873
Epoch 56/100
624/624 [==============================] - 0s 541us/step - loss: 0.3073 - acc: 0.9760 - val_loss: 0.1953 - val_acc: 0.9873
Epoch 57/100
624/624 [==============================] - 0s 529us/step - loss: 0.2835 - acc: 0.9728 - val_loss: 0.1859 - val_acc: 0.9873
Epoch 58/100
624/624 [==============================] - 0s 531us/step - loss: 0.2768 - acc: 0.9744 - val_loss: 0.1768 - val_acc: 0.9873
Epoch 59/100
624/624 [==============================] - 0s 533us/step - loss: 0.2660 - acc: 0.9712 - val_loss: 0.1682 - val_acc: 0.9936
Epoch 60/100
624/624 [==============================] - 0s 531us/step - loss: 0.2662 - acc: 0.9728 - val_loss: 0.1601 - val_acc: 0.9936
Epoch 61/100
624/624 [==========================

Processed 004 of class Swing-Bench
Processed 016 of class Swing-Bench
Processed 001 of class Swing-Bench
Processed 007 of class Swing-Bench
Processed 008 of class Swing-Bench
Processed 002 of class Swing-Bench
Processed 017 of class Swing-Bench
Processed 015 of class Swing-Bench
Processed 005 of class Lifting
Processed 003 of class Lifting
Processed 004 of class Lifting
Processed 001 of class Lifting
Processed 002 of class Lifting
Processed 009 of class Swing-Side
Processed 005 of class Swing-Side
Processed 011 of class Swing-Side
Processed 010 of class Swing-Side
Processed 003 of class Swing-Side
Processed 006 of class Swing-Side
Processed 004 of class Swing-Side
Processed 001 of class Swing-Side
Processed 007 of class Swing-Side
Processed 008 of class Swing-Side
Processed 002 of class Swing-Side
Processed 014 of class Walking
Processed 009 of class Walking
Processed 005 of class Walking
Processed 011 of class Walking
Processed 010 of class Walking
Processed 018 of class Walking
Proce

...

Epoch 100/100
321/321 [==============================] - 0s 611us/step - loss: 0.1137 - acc: 0.9969 - val_loss: 0.2147 - val_acc: 0.9630
Processed 014 of class Kicking
Processed 009 of class Kicking
Processed 005 of class Kicking
Processed 011 of class Kicking
Processed 010 of class Kicking
Processed 003 of class Kicking
Processed 012 of class Kicking
Processed 006 of class Kicking
Processed 013 of class Kicking
Processed 004 of class Kicking
Processed 016 of class Kicking
Processed 001 of class Kicking
Processed 007 of class Kicking
Processed 002 of class Kicking
Processed 017 of class Kicking
Processed 015 of class Kicking
Processed 009 of class Riding-Horse
Processed 005 of class Riding-Horse
Processed 010 of class Riding-Horse
Processed 003 of class Riding-Horse
Processed 006 of class Riding-Horse
Processed 004 of class Riding-Horse
Processed 001 of class Riding-Horse
Processed 007 of class Riding-Horse
Processed 008 of class Riding-Horse
Processed 002 of class Riding-Horse
Process

Epoch 33/100
257/257 [==============================] - 0s 817us/step - loss: 0.7851 - acc: 0.9105 - val_loss: 0.8754 - val_acc: 0.8615
Epoch 34/100
257/257 [==============================] - 0s 822us/step - loss: 0.7701 - acc: 0.9027 - val_loss: 0.8502 - val_acc: 0.8615
Epoch 35/100
257/257 [==============================] - 0s 792us/step - loss: 0.7363 - acc: 0.8949 - val_loss: 0.8210 - val_acc: 0.8615
Epoch 36/100
257/257 [==============================] - 0s 824us/step - loss: 0.7035 - acc: 0.9144 - val_loss: 0.7893 - val_acc: 0.8615
Epoch 37/100
257/257 [==============================] - 0s 803us/step - loss: 0.6717 - acc: 0.9144 - val_loss: 0.7600 - val_acc: 0.8615
Epoch 38/100
257/257 [==============================] - 0s 807us/step - loss: 0.6599 - acc: 0.9144 - val_loss: 0.7326 - val_acc: 0.8615
Epoch 39/100
257/257 [==============================] - 0s 810us/step - loss: 0.6246 - acc: 0.9027 - val_loss: 0.7067 - val_acc: 0.8615
Epoch 40/100
257/257 [==========================

Epoch 94/100
257/257 [==============================] - 0s 833us/step - loss: 0.1554 - acc: 0.9961 - val_loss: 0.3238 - val_acc: 0.9231
Epoch 95/100
257/257 [==============================] - 0s 849us/step - loss: 0.1430 - acc: 1.0000 - val_loss: 0.3240 - val_acc: 0.9231
Epoch 96/100
257/257 [==============================] - 0s 857us/step - loss: 0.1441 - acc: 0.9961 - val_loss: 0.3247 - val_acc: 0.9231
Epoch 97/100
257/257 [==============================] - 0s 876us/step - loss: 0.1371 - acc: 1.0000 - val_loss: 0.3264 - val_acc: 0.9385
Epoch 98/100
257/257 [==============================] - 0s 855us/step - loss: 0.1329 - acc: 0.9961 - val_loss: 0.3262 - val_acc: 0.9385
Epoch 99/100
257/257 [==============================] - 0s 845us/step - loss: 0.1320 - acc: 1.0000 - val_loss: 0.3259 - val_acc: 0.9385
Epoch 100/100
257/257 [==============================] - 0s 862us/step - loss: 0.1310 - acc: 1.0000 - val_loss: 0.3243 - val_acc: 0.9385
Processed 014 of class Kicking
Processed 009 of

Epoch 27/100
203/203 [==============================] - 0s 1ms/step - loss: 1.1710 - acc: 0.7931 - val_loss: 1.1765 - val_acc: 0.7843
Epoch 28/100
203/203 [==============================] - 0s 1ms/step - loss: 1.1398 - acc: 0.8276 - val_loss: 1.1415 - val_acc: 0.7843
Epoch 29/100
203/203 [==============================] - 0s 1ms/step - loss: 1.0955 - acc: 0.8325 - val_loss: 1.1061 - val_acc: 0.7843
Epoch 30/100
203/203 [==============================] - 0s 1ms/step - loss: 1.0605 - acc: 0.8177 - val_loss: 1.0677 - val_acc: 0.7843
Epoch 31/100
203/203 [==============================] - 0s 1ms/step - loss: 0.9995 - acc: 0.8522 - val_loss: 1.0339 - val_acc: 0.7843
Epoch 32/100
203/203 [==============================] - 0s 1ms/step - loss: 0.9753 - acc: 0.8522 - val_loss: 1.0007 - val_acc: 0.8039
Epoch 33/100
203/203 [==============================] - 0s 1ms/step - loss: 0.9610 - acc: 0.8424 - val_loss: 0.9597 - val_acc: 0.8235
Epoch 34/100
203/203 [==============================] - 0s 1ms

Epoch 89/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1734 - acc: 0.9901 - val_loss: 0.2193 - val_acc: 0.9608
Epoch 90/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1728 - acc: 0.9951 - val_loss: 0.2158 - val_acc: 0.9608
Epoch 91/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1503 - acc: 0.9951 - val_loss: 0.2121 - val_acc: 0.9608
Epoch 92/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1592 - acc: 0.9951 - val_loss: 0.2095 - val_acc: 0.9608
Epoch 93/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1635 - acc: 0.9951 - val_loss: 0.2077 - val_acc: 0.9608
Epoch 94/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1350 - acc: 0.9901 - val_loss: 0.2069 - val_acc: 0.9608
Epoch 95/100
203/203 [==============================] - 0s 2ms/step - loss: 0.1379 - acc: 0.9951 - val_loss: 0.2069 - val_acc: 0.9608
Epoch 96/100
203/203 [==============================] - 0s 2ms

In [61]:
print(histories['11_20'].history)

{'val_loss': [2.2832119464874268, 2.2261219024658203, 2.172034978866577, 2.121187686920166, 2.0728635787963867, 2.026477813720703, 1.9819071292877197, 1.9384453296661377, 1.895646333694458, 1.8536524772644043, 1.8127917051315308, 1.7724930047988892, 1.7319284677505493, 1.6911121606826782, 1.6507118940353394, 1.6110646724700928, 1.571265697479248, 1.5312467813491821, 1.4904510974884033, 1.4488615989685059, 1.4062772989273071, 1.3633582592010498, 1.3198163509368896, 1.2760250568389893, 1.2317285537719727, 1.1864784955978394, 1.1415950059890747, 1.0977016687393188, 1.0558370351791382, 1.015629768371582, 0.977288007736206, 0.9404245018959045, 0.9043866991996765, 0.8701678514480591, 0.836543083190918, 0.8044388294219971, 0.7743943929672241, 0.7464659214019775, 0.7198847532272339, 0.6946978569030762, 0.6706190705299377, 0.647598147392273, 0.6256670355796814, 0.6043695211410522, 0.5836827754974365, 0.5638894438743591, 0.5445824861526489, 0.5252348184585571, 0.5061926245689392, 0.4874929487705

In [62]:
import pickle
with open("results.pkl", "wb") as f:
    pickle.dump(results, f)

In [73]:
with open("histories.pkl", "rb") as f:
    histories2  = pickle.load(f)

In [77]:
histories['5_20'].history

{'acc': [0.18692810833454132,
  0.265359491109848,
  0.3150326907634735,
  0.3986928164958954,
  0.46405228972435,
  0.5084967613220215,
  0.5346405506134033,
  0.5856209397315979,
  0.6026144027709961,
  0.6326797604560852,
  0.6562091708183289,
  0.6901960968971252,
  0.7058823704719543,
  0.7176470756530762,
  0.7137255072593689,
  0.7202614545822144,
  0.7424836754798889,
  0.7503268122673035,
  0.7529411911964417,
  0.758169949054718,
  0.7803921699523926,
  0.7816993594169617,
  0.7934640645980835,
  0.7921568751335144,
  0.8039215803146362,
  0.8196078538894653,
  0.8209150433540344,
  0.8418300747871399,
  0.8339869379997253,
  0.843137264251709,
  0.8588235378265381,
  0.8679738640785217,
  0.8797385692596436,
  0.8797385692596436,
  0.8797385692596436,
  0.8980392217636108,
  0.9019607901573181,
  0.9111111164093018,
  0.8993464112281799,
  0.9084967374801636,
  0.9071895480155945,
  0.9215686321258545,
  0.9189542531967163,
  0.9241830110549927,
  0.9241830110549927,
  0.926

In [42]:
history = rnn.fit(X_train_rnn, Y_train_rnn, epochs=5, batch_size = X_train_rnn.shape[0], validation_split=0.2)

Train on 364 samples, validate on 92 samples
Epoch 1/5
364/364 [==============================] - 0s 660us/step - loss: 0.0677 - acc: 1.0000 - val_loss: 0.1058 - val_acc: 0.9783
Epoch 2/5
364/364 [==============================] - 0s 662us/step - loss: 0.0738 - acc: 0.9973 - val_loss: 0.1052 - val_acc: 0.9783
Epoch 3/5
364/364 [==============================] - 0s 909us/step - loss: 0.0645 - acc: 1.0000 - val_loss: 0.1051 - val_acc: 0.9783
Epoch 4/5
364/364 [==============================] - 0s 712us/step - loss: 0.0659 - acc: 1.0000 - val_loss: 0.1052 - val_acc: 0.9783
Epoch 5/5
364/364 [==============================] - 0s 673us/step - loss: 0.0655 - acc: 1.0000 - val_loss: 0.1058 - val_acc: 0.9783


In [43]:
#rnn = load_model('models/LSTM_Strided/500ep_valacc_97_single_LSTM.h5')

print(X_test.shape)
print(Y_test.shape)

evaluate(X_test,Y_test, rnn)

(14,)
(14, 9)
Max Preds time [8]
Pred Golf-Swing Actual Kicking

Max Preds time [0]
Pred Kicking Actual Kicking

Max Preds time [1, 1, 1, 1]
Pred Riding-Horse Actual Riding-Horse

Max Preds time [7, 7]
Pred Walking Actual Riding-Horse

Max Preds time [1, 1, 1, 1, 1]
Pred Riding-Horse Actual Running

Max Preds time [4, 4, 4, 4, 4]
Pred Swing-Bench Actual SkateBoarding

Max Preds time [4, 4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [4, 4, 4]
Pred Swing-Bench Actual Swing-Bench

Max Preds time [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Pred Lifting Actual Lifting

Max Preds time [6]
Pred Swing-Side Actual Swing-Side

Max Preds time [7, 7, 7, 5]
Pred Walking Actual Walking

Max Preds time [7, 7, 7, 7, 7, 7, 7, 7, 2]
Pred Walking Actual Walking

Max Preds time [8]
Pred Golf-Swing Actual Golf-Swing

Max Preds time [8, 8, 8, 8]
Pred Golf-Swing Actual Golf-Swing



71.42857142857143

In [130]:
rnn.save('models/LSTM_Strided/100ep_valacc_94_double_LSTM_dropout_10_30.h5')

## Testing

In [30]:
def build_test_dataset(image_size, stride = 10, max_len = 40):
    
    model = load_model('models/Conv/17epochs_valacc_94.h5')
    
    X_test_images = []
    Y_test_images = []
    
    VIDEOS_DIR = './UCF_Unseen/'
    IMAGES_DIR = './UCF_Images/'
    
    videos = []
    for x in classes:
        videos.append(list(os.listdir(VIDEOS_DIR+x+'/')))
    
    for i in range(len(classes)):
        cls = classes[i]

        for j in range(len(videos[i])):
            vid = videos[i][j]
            video_r = VIDEOS_DIR+cls+'/'+ vid +'/'
            image_r = IMAGES_DIR+cls+'/'+ vid +'/'
            
            filelist = sorted(list(os.listdir(image_r)))
            X_train_images_class = []
            
            for file in filelist:
                if file.endswith(".png"):
                    image = load_image(image_r+file,image_size)
                    X_train_images_class.append(image)
            X_cnn = model_predict(model,np.array(X_train_images_class))
            print(X_cnn.shape)
            
            del X_train_images_class
            X_test_frames = []                                
            for k in range(0,X_cnn.shape[0],stride):
                lower = k
                upper = min(X_cnn.shape[0],k+max_len)
                if upper == X_cnn.shape[0]:
                    X_test_frames.append(pad(X_cnn[lower:upper],max_len))
                    X_test_images.append(np.array(X_test_frames))        
                    Y_test_images.append(i)
                    print("Padded frames" , lower , "to" , upper)
                    break
                else:
                    X_test_frames.append(X_cnn[lower:upper])
                    print("Added frames" , lower , "to" , upper)
                    
            print("Processed",videos[i][j],"of","class",classes[i])

    return X_test_images,Y_test_images

In [32]:
X_test,Y_test = build_test_dataset((172,172))

(22, 128)
Padded frames 0 to 22
Processed 010 of class Kicking
(23, 128)
Padded frames 0 to 23
Processed 006 of class Kicking
(23, 128)
Padded frames 0 to 23
Processed 004 of class Kicking
(20, 128)
Padded frames 0 to 20
Processed 008 of class Kicking
(58, 128)
Added frames 0 to 40
Added frames 10 to 50
Padded frames 20 to 58
Processed 011 of class Riding-Horse
(36, 128)
Padded frames 0 to 36
Processed 012 of class Riding-Horse
(65, 128)
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Padded frames 30 to 65
Processed 012 of class Running
(70, 128)
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Padded frames 30 to 70
Processed 011 of class SkateBoarding
(70, 128)
Added frames 0 to 40
Added frames 10 to 50
Added frames 20 to 60
Padded frames 30 to 70
Processed 012 of class SkateBoarding
(50, 128)
Added frames 0 to 40
Padded frames 10 to 50
Processed 018 of class Swing-Bench
(50, 128)
Added frames 0 to 40
Padded frames 10 to 50
Processed 020 of class Swi

In [34]:
X_test = np.array(X_test)
[i.shape for i in X_test]

[(1, 40, 128),
 (1, 40, 128),
 (1, 40, 128),
 (1, 40, 128),
 (3, 40, 128),
 (1, 40, 128),
 (4, 40, 128),
 (4, 40, 128),
 (4, 40, 128),
 (2, 40, 128),
 (2, 40, 128),
 (2, 40, 128),
 (10, 40, 128),
 (5, 40, 128),
 (1, 40, 128),
 (8, 40, 128),
 (5, 40, 128),
 (8, 40, 128),
 (3, 40, 128),
 (3, 40, 128),
 (3, 40, 128),
 (3, 40, 128)]

In [37]:
Y_test = convert_to_one_hot(np.array(Y_test), 9)
Y_test.shape

(22, 9)

In [49]:
evaluate(X_test,Y_test,rnn)

Max Preds time [0]
Pred Kicking Actual Kicking
Max Preds time [0]
Pred Kicking Actual Kicking
Max Preds time [2]
Pred Running Actual Kicking
Max Preds time [7]
Pred Walking Actual Kicking
Max Preds time [1, 1, 1]
Pred Riding-Horse Actual Riding-Horse
Max Preds time [1]
Pred Riding-Horse Actual Riding-Horse
Max Preds time [8, 8, 8, 8]
Pred Golf-Swing Actual Running
Max Preds time [0, 3, 3, 3]
Pred SkateBoarding Actual SkateBoarding
Max Preds time [2, 2, 2, 2]
Pred Running Actual SkateBoarding
Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench
Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench
Max Preds time [4, 4]
Pred Swing-Bench Actual Swing-Bench
Max Preds time [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Pred Lifting Actual Lifting
Max Preds time [6, 6, 6, 6, 6]
Pred Swing-Side Actual Swing-Side
Max Preds time [6]
Pred Swing-Side Actual Swing-Side
Max Preds time [7, 7, 7, 7, 7, 7, 7, 7]
Pred Walking Actual Walking
Max Preds time [7, 7, 7, 7, 7]
Pred Walking Actual Walking
Max Preds 

81.81818181818183